In [2]:
import ee
try:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project = 'ee-gsingh')
except: 
    ee.Authenticate()
    ee.Initialize()

import geemap
from geeml.utils import eeprint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
pts = ee.FeatureCollection('projects/ee-geethensingh/assets/postdoc/aliens_sep2018_bioscape2023')
eeprint(pts.limit(5))

In [4]:
class prepareTrainingData:
    """This class prepares training data for the earth engine random forest model."""
    def __init__(self, points: ee.FeatureCollection, targetProperty: str, nFolds: int, proj: str = 'EPSG:4326'):
        """
        Args:
            points (ee.FeatureCollection): ee.FeatureCollection
            targetProperty (str): name of the property that contains the target variable
            nFolds (int): number of folds for cross validation
            proj (str): projection of the data
        """

        self.points = points
        self.targetProperty = targetProperty
        self.nFolds = nFolds
        self.proj = proj

    def addCoordProperty(self, features: ee.FeatureCollection) -> ee.FeatureCollection:
        """
        This function adds coordinates to the points

        Args:
            features (ee.FeatureCollection): ee.FeatureCollection

        Returns:
            ee.FeatureCollection: ee.FeatureCollection
        """
        def coords(feature):
            return feature.geometry().transform(proj = self.proj).coordinates()

        return features.map(lambda ft: ft.set('x', coords(ft).getNumber(0)).set('y', coords(ft).getNumber(1)))
        
    def _preparePoints(self):
        # add coordinates to points.
        points = self.addCoordProperty(self.points)

        # cluster points into groups based on coordinates
        clusterer = ee.Clusterer.wekaKMeans(self.nFolds).train(features = points,inputProperties = ['x','y'])
        points = points.cluster(clusterer)
        return points

In [4]:
wKfold = prepareTrainingData(points=pts, targetProperty='class', nFolds=5)._preparePoints()
eeprint(wKfold.limit(5))

In [5]:
# Create a color palette for unique 'group' values
group_values = wKfold.aggregate_array('cluster').distinct()
palette = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'black', 'white']

# Create a dictionary to map each group to a color
group_list = group_values.getInfo()
color_dict = {group: palette[i % len(palette)] for i, group in enumerate(group_list)}

# Create a list of styled FeatureCollections for each group
styled_fc_list = []
for group in group_list:
    color = color_dict[group]
    fc_group = wKfold.filter(ee.Filter.eq('cluster', group))
    styled_fc = fc_group.map(lambda f: f.set('style', {
        'color': color,
        'pointSize': 4,
        'width': 1
    }))
    styled_fc_list.append(styled_fc)

# Combine all styled groups into one
styled_pts = ee.FeatureCollection(styled_fc_list).flatten()

# Set up the map
Map = geemap.Map(center=[-30, 25], zoom=6)
Map.addLayer(styled_pts.style(**{'styleProperty': 'style'}), {}, 'Points by Group')
Map

Map(center=[-30, 25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [1]:
def createComposite(imageCollection: str, points: ee.Geometry, year: int, period:str):
    """Creates a composite image of a point over a given year (uses 1 month period for 2018 and 2 months for 2023).
    
    Args:
        imageCollection (str): The collection to use for the composite.
        point (ee.Geometry): The point to create the composite for.
        year (int): The year to create the composite for.
        period (str): Either values of first (Sept-09), second(oct-Nov) or both. 
    
    Returns:
        ee.Image: The composite image.
    """

    # Get the image collection
    ic = ee.ImageCollection(imageCollection)

    if period == 'first':
        startDate = f'{year}-09-01'
        endDate = f'{year}-10-01'
    elif period == 'second':
        startDate = f'{year}-10-22'
        endDate = f'{year}-11-27'
        # startDate = f'{year}-10-01'
        # endDate = f'{year}-12-01'
        

    # Mask clouds using cloud score+
    csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')

    QA_BAND = 'cs_cdf'
    CLEAR_THRESHOLD = 0.65
    
    # Filter the collection to the start and end dates, and point
    medianImage = ic.filterDate(startDate, endDate).filterBounds(points).linkCollection(csPlus, [QA_BAND])\
    .map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))\
    .median()
    
    return medianImage.divide(10000)

NameError: name 'ee' is not defined

In [7]:
# ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

composite2018 = createComposite(imageCollection= ee.ImageCollection("COPERNICUS/S2_HARMONIZED"),
                points=pts.geometry(),
                year=2018,
                period='first')

# Create a composite for 2023
composite2023 = createComposite(imageCollection= ee.ImageCollection("COPERNICUS/S2_HARMONIZED"),
                points=pts.geometry(),
                year=2023,
                period='second')

In [8]:
# Centre map on South Africa, western Cape
Map = geemap.Map(center=[-33.86, 19.21], zoom=8)
Map.addLayer(pts, {}, 'Points')
Map.addLayer(composite2018, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Composite 2018')
Map.addLayer(composite2023, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Composite 2023')
Map

Map(center=[-33.86, 19.21], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [9]:
def getSpectralSignature(image: ee.Image, point: ee.Geometry, scale: int = 10):
    """Plots the spectral signature of a point on an image.
    
    Args:
        image (ee.Image): The image to plot the spectral signature on.
        point (ee.Geometry): The point to plot the spectral signature for.
        includeMap (bool, optional): Whether to include a map of the image. Defaults to True.
        scale (int, optional): The scale to plot the spectral signature at. Defaults to None.
    """

    # Get the spectral signature of the point
    spectralSignature = image.select('B.*').reduceRegion(reducer = ee.Reducer.first()
                                                        , geometry = point
                                                        , scale = scale)

    # Convert to geodataframe
    pts_gdf = ee.data.computeFeatures({
        'expression': spectralSignature,
        'fileFormat': 'PANDAS_DATAFRAME'
    })
    
    return pts_gdf

In [10]:
# extract values at points
data = composite2018.reduceRegions(collection=wKfold, reducer=ee.Reducer.first(), scale=10)
data.limit(5)

# convert to geopandas dataframe
gdf18 = ee.data.computeFeatures({
    'expression': data,
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})

# Need to set the CRS.
# Make sure it matches the CRS of FeatureCollection geometries.
gdf18.crs = 'EPSG:4326'

gdf18.head()

# extract values at points
data = composite2023.reduceRegions(collection=wKfold, reducer=ee.Reducer.first(), scale=10)
data.limit(5)

# convert to geopandas dataframe
gdf23 = ee.data.computeFeatures({
    'expression': data,
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})

# Need to set the CRS.
# Make sure it matches the CRS of FeatureCollection geometries.
gdf23.crs = 'EPSG:4326'

gdf23.head()

,geometry,2018_2023,B1,B10,B11,B12,B2,B3,B4,B5,...,cluster,cs_cdf,fid,group,layer,notes,path,x,y,2023_class
0,POINT (19.07261 -33.80554),0.006703,0.1558,0.00140,0.36630,0.27330,0.16290,0.17400,0.1985,0.2294,...,3,0.000089,1028,0,valid,,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.072610,-33.805538,NaN
1,POINT (19.07795 -33.80792),0.012548,0.1129,0.00100,0.03810,0.02360,0.08830,0.07860,0.0741,0.0667,...,3,0.000091,1029,0,valid,chngg rb,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.077947,-33.807923,NaN
2,POINT (19.07821 -33.80821),0.021821,0.1129,0.00100,0.02240,0.01270,0.09460,0.09050,0.0723,0.0605,...,3,0.000091,1030,0,valid,chngg rb,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.078206,-33.808209,NaN
3,POINT (19.14952 -33.7358),0.021899,0.1313,0.00135,0.26125,0.17635,0.18115,0.17685,0.1839,0.1800,...,3,0.000091,1125,0,valid,,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.149525,-33.735797,NaN
4,POINT (19.26657 -33.75278),0.003018,0.1426,0.00150,0.27870,0.21050,0.14290,0.13580,0.1425,0.1617,...,1,0.000087,1139,0,valid,,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.266572,-33.752777,NaN


### Models

### Experiment 1: train on 2018 (all), predict on 2018 (all)

In [11]:
gdf18.columns

wavelength23_cols = ['B1', 'B10', 'B11', 'B12', 'B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9']

In [13]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.46, F1 Score: 0.45
              precision    recall  f1-score   support

           0       0.55      0.50      0.52        12
           1       0.00      0.00      0.00         0
           2       0.25      0.47      0.33        32
           3       0.38      0.74      0.50        78
           4       0.61      0.62      0.61        65
           5       0.91      0.26      0.41        38
           6       0.86      0.35      0.49        52
           7       0.67      0.27      0.38        37
           8       0.50      0.19      0.27        91
           9       0.57      0.80      0.67        25
          10       0.36      1.00      0.53         8

    accuracy                           0.46       438
   macro avg       0.51      0.47      0.43       438
weighted avg       0.57      0.46      0.45       438


Fold 2 — Accuracy: 0.51, F1 Score: 0.54
              precision    recall  f1-score   support

           0       1.00      0.60      0.75       

In [50]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Define features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']

# CatBoost model
clf = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='MultiClass',
    verbose=0,  # Change to 100 if you want to monitor training
    random_seed=42
)

# GroupKFold setup
n_splits = 3
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # CatBoost needs Pool objects for training when using advanced features
    train_pool = Pool(X_train, label=y_train)
    test_pool = Pool(X_test, label=y_test)
    
    clf.fit(train_pool)
    y_pred = clf.predict(test_pool).astype(int).flatten()

    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

    print(f"\nFold {i + 1} — F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Print average F1
print(f"\nAverage F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f}")



Fold 1 — F1 Score: 0.33
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        21
           1       0.17      0.08      0.11        24
           2       0.23      0.38      0.29        60
           3       0.42      0.60      0.50       147
           4       0.33      0.63      0.44       103
           5       0.39      0.29      0.33        42
           6       0.35      0.16      0.22        74
           7       0.32      0.45      0.37        67
           8       0.42      0.22      0.29       126
           9       0.00      0.00      0.00       115
          10       0.64      1.00      0.78        30

    accuracy                           0.38       809
   macro avg       0.36      0.41      0.36       809
weighted avg       0.33      0.38      0.33       809


Fold 2 — F1 Score: 0.50
              precision    recall  f1-score   support

           0       1.00      0.56      0.71        18
           1       0.00     

In [32]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = gdf18[wavelength23_cols], gdf18['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [33]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .64

Accuracy: 0.64


In [ ]:
!uv pip install "tabpfn-extensions[all] @ git+https://github.com/PriorLabs/tabpfn-extensions.git"

In [27]:
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']  # ensure this column exists

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Print average F1
print(f"\nAverage F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [01:12<00:00,  4.56s/it]



Fold 1 — F1 Score: 0.63
              precision    recall  f1-score   support

           0       1.00      0.75      0.86        12
           1       0.00      0.00      0.00         0
           2       0.42      0.56      0.48        32
           3       0.52      0.77      0.62        78
           4       0.69      0.71      0.70        65
           5       0.74      0.37      0.49        38
           6       0.84      0.73      0.78        52
           7       0.86      0.51      0.64        37
           8       0.68      0.45      0.54        91
           9       0.63      0.88      0.73        25
          10       0.50      1.00      0.67         8

    accuracy                           0.63       438
   macro avg       0.63      0.61      0.59       438
weighted avg       0.68      0.63      0.63       438



100%|██████████| 16/16 [01:12<00:00,  4.55s/it]



Fold 2 — F1 Score: 0.66
              precision    recall  f1-score   support

           0       1.00      0.73      0.85        15
           1       0.00      0.00      0.00         0
           2       0.84      0.78      0.81       124
           3       0.25      0.26      0.25        39
           4       0.91      0.59      0.72        49
           5       0.55      0.67      0.61        46
           6       0.58      0.68      0.62        22
           7       0.64      0.82      0.72        11
           8       0.42      0.50      0.45        50
           9       0.00      0.00      0.00         0
          10       1.00      0.90      0.95        20

    accuracy                           0.65       376
   macro avg       0.56      0.54      0.54       376
weighted avg       0.69      0.65      0.66       376



100%|██████████| 16/16 [01:15<00:00,  4.71s/it]



Fold 3 — F1 Score: 0.65
              precision    recall  f1-score   support

           0       0.69      0.87      0.77        23
           1       0.33      0.05      0.09        19
           2       0.59      0.62      0.61        16
           3       0.84      0.76      0.80       102
           4       0.64      0.63      0.64        65
           5       0.40      0.29      0.33        14
           6       0.15      0.21      0.18        14
           7       0.50      0.79      0.61        24
           8       0.42      0.56      0.48         9
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.98        21

    accuracy                           0.65       307
   macro avg       0.51      0.52      0.50       307
weighted avg       0.66      0.65      0.65       307



100%|██████████| 16/16 [01:18<00:00,  4.88s/it]



Fold 4 — F1 Score: 0.78
              precision    recall  f1-score   support

           0       1.00      0.78      0.88         9
           1       0.00      0.00      0.00         0
           2       0.87      0.78      0.83        79
           3       0.80      0.75      0.78        65
           4       0.72      0.74      0.73        39
           5       0.00      0.00      0.00         2
           6       0.73      0.73      0.73        26
           7       0.47      0.64      0.54        11
           8       0.52      0.79      0.62        19
           9       0.88      0.83      0.85        42
          10       0.92      1.00      0.96        12

    accuracy                           0.77       304
   macro avg       0.63      0.64      0.63       304
weighted avg       0.79      0.77      0.78       304



100%|██████████| 16/16 [01:22<00:00,  5.13s/it]


Fold 5 — F1 Score: 0.67
              precision    recall  f1-score   support

           0       0.82      0.93      0.88        15
           1       1.00      0.12      0.22        24
           2       0.75      0.53      0.62        17
           3       0.81      0.88      0.84        43
           4       0.52      0.71      0.60        34
           5       0.00      0.00      0.00         3
           6       0.12      0.40      0.19         5
           7       0.62      0.70      0.66        30
           8       0.68      0.61      0.64        28
           9       0.91      0.67      0.77        48
          10       1.00      1.00      1.00        16

    accuracy                           0.67       263
   macro avg       0.66      0.60      0.58       263
weighted avg       0.76      0.67      0.67       263


Average F1 Score over 5 folds: 0.68 (0.05)


In [25]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = gdf18[wavelength23_cols], gdf18['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ModuleNotFoundError: No module named 'tabpfn_extensions'

In [ ]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

#from 0.813

100%|██████████| 16/16 [00:05<00:00,  3.05it/s]

Accuracy 0.7843601895734598


### Experiment 1.5: train on 2018 (no change), predict on 2018 (no change)

In [14]:
df18subset = gdf18[gdf18['change'] == 0].copy()

In [15]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = df18subset[wavelength23_cols]
y = df18subset['class'].astype(int)
groups = df18subset['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.41, F1 Score: 0.41
              precision    recall  f1-score   support

           0       0.67      0.33      0.44        12
           1       0.00      0.00      0.00         0
           2       0.09      0.27      0.14        15
           3       0.32      0.69      0.44        64
           4       0.49      0.53      0.51        58
           5       0.80      0.35      0.49        34
           6       0.76      0.25      0.38        51
           7       0.71      0.41      0.52        37
           8       0.59      0.21      0.31        90
          10       0.35      1.00      0.52         8

    accuracy                           0.41       369
   macro avg       0.48      0.40      0.37       369
weighted avg       0.56      0.41      0.41       369


Fold 2 — Accuracy: 0.55, F1 Score: 0.54
              precision    recall  f1-score   support

           0       0.80      0.87      0.83        23
           1       0.67      0.11      0.18       

In [37]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df18subset[wavelength23_cols], df18subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [38]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .65

Accuracy: 0.63


In [17]:
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']  # replace with the appropriate grouping column

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


  0%|          | 0/16 [00:00<?, ?it/s]

(…)fn-v2-classifier-finetuned-zk73skhh.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

100%|██████████| 16/16 [01:18<00:00,  4.89s/it]



Fold 1 — Accuracy: 0.62, F1 Score: 0.63
              precision    recall  f1-score   support

           0       0.90      0.75      0.82        12
           1       0.00      0.00      0.00         0
           2       0.40      0.56      0.47        32
           3       0.53      0.71      0.61        78
           4       0.65      0.71      0.68        65
           5       0.76      0.42      0.54        38
           6       0.83      0.73      0.78        52
           7       0.91      0.54      0.68        37
           8       0.66      0.44      0.53        91
           9       0.64      0.92      0.75        25
          10       0.53      1.00      0.70         8

    accuracy                           0.62       438
   macro avg       0.62      0.62      0.59       438
weighted avg       0.67      0.62      0.63       438



...


Fold 3 — Accuracy: 0.66, F1 Score: 0.65
              precision    recall  f1-score   support

           0       0.69      0.87      0.77        23
           1       0.50      0.05      0.10        19
           2       0.60      0.75      0.67        16
           3       0.84      0.73      0.78       102
           4       0.63      0.66      0.65        65
           5       0.42      0.36      0.38        14
           6       0.21      0.29      0.24        14
           7       0.53      0.79      0.63        24
           8       0.42      0.56      0.48         9
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.98        21

    accuracy                           0.66       307
   macro avg       0.53      0.55      0.52       307
weighted avg       0.68      0.66      0.65       307



100%|██████████| 16/16 [01:19<00:00,  4.95s/it]



Fold 4 — Accuracy: 0.77, F1 Score: 0.78
              precision    recall  f1-score   support

           0       1.00      0.78      0.88         9
           1       0.00      0.00      0.00         0
           2       0.88      0.80      0.83        79
           3       0.81      0.74      0.77        65
           4       0.71      0.74      0.72        39
           5       0.00      0.00      0.00         2
           6       0.70      0.73      0.72        26
           7       0.46      0.55      0.50        11
           8       0.52      0.74      0.61        19
           9       0.86      0.86      0.86        42
          10       0.92      1.00      0.96        12

    accuracy                           0.77       304
   macro avg       0.62      0.63      0.62       304
weighted avg       0.79      0.77      0.78       304



100%|██████████| 16/16 [01:23<00:00,  5.19s/it]


Fold 5 — Accuracy: 0.68, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.82      0.93      0.88        15
           1       0.80      0.17      0.28        24
           2       0.75      0.53      0.62        17
           3       0.81      0.91      0.86        43
           4       0.54      0.74      0.62        34
           5       0.00      0.00      0.00         3
           6       0.12      0.40      0.18         5
           7       0.62      0.70      0.66        30
           8       0.71      0.61      0.65        28
           9       0.91      0.67      0.77        48
          10       1.00      1.00      1.00        16

    accuracy                           0.68       263
   macro avg       0.64      0.60      0.59       263
weighted avg       0.75      0.68      0.69       263


Average Accuracy over 5 folds: 0.68 (0.05)
Average F1 Score over 5 folds: 0.68 (0.05)


In [39]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df18subset[wavelength23_cols], df18subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [40]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .785

Accuracy 0.7417503586800573


### Experiment 1.6: train on 2023 (all), predict on 2023 (all)

In [19]:
# use 2018 class for 2023 whereever there is no change
gdf23.loc[gdf23['change'] == 0, '2023_class'] = gdf23['class']
gdf23.dropna(subset=['2023_class'], inplace=True)

In [21]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
groups = gdf23['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.42      0.64      0.51        25
           1       0.00      0.00      0.00         0
           2       0.10      0.33      0.15        15
           3       0.48      0.61      0.53        64
           4       0.59      0.64      0.61        84
           5       0.76      0.38      0.51        34
           6       0.78      0.35      0.49        51
           7       0.62      0.74      0.67        46
           8       0.57      0.33      0.42        91
          10       0.36      1.00      0.53         8
          12       0.00      0.00      0.00        18

    accuracy                           0.50       436
   macro avg       0.43      0.46      0.40       436
weighted avg       0.55      0.50      0.50       436


Fold 2 — Accuracy: 0.44, F1 Score: 0.44
              precision    recall  f1-score   support

           0       0.50      0.25      0.33       

In [42]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [44]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .54

Accuracy: 0.59


In [22]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
groups = gdf23['cluster']  # replace with the appropriate grouping column

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [01:12<00:00,  4.53s/it]



Fold 1 — Accuracy: 0.58, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.38      0.44      0.41        25
           1       0.00      0.00      0.00         0
           2       0.24      0.53      0.33        15
           3       0.54      0.69      0.60        64
           4       0.60      0.65      0.62        84
           5       0.90      0.56      0.69        34
           6       0.88      0.69      0.77        51
           7       0.64      0.78      0.71        46
           8       0.64      0.41      0.50        91
          10       0.40      1.00      0.57         8
          12       0.00      0.00      0.00        18

    accuracy                           0.58       436
   macro avg       0.47      0.52      0.47       436
weighted avg       0.61      0.58      0.58       436



100%|██████████| 16/16 [01:17<00:00,  4.84s/it]



Fold 2 — Accuracy: 0.53, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.48      0.19      0.27        52
           1       0.00      0.00      0.00         0
           2       0.66      0.57      0.61        68
           3       0.26      0.52      0.34        31
           4       0.30      0.66      0.41        32
           5       0.65      0.67      0.66        45
           6       0.77      0.91      0.83        22
           7       0.79      0.41      0.54        27
           8       0.64      0.57      0.61        47
          10       0.96      0.89      0.92        27
          12       0.00      0.00      0.00        25

    accuracy                           0.53       376
   macro avg       0.50      0.49      0.47       376
weighted avg       0.56      0.53      0.52       376



100%|██████████| 16/16 [01:21<00:00,  5.07s/it]



Fold 3 — Accuracy: 0.59, F1 Score: 0.59
              precision    recall  f1-score   support

           0       0.33      0.80      0.47        25
           1       0.00      0.00      0.00        19
           2       0.43      0.75      0.55        16
           3       0.89      0.52      0.66       103
           4       0.66      0.55      0.60        60
           5       0.41      0.64      0.50        14
           6       0.60      0.64      0.62        14
           7       0.71      0.83      0.77        24
           8       0.50      0.56      0.53         9
          10       0.91      1.00      0.95        20
          12       0.00      0.00      0.00         3

    accuracy                           0.59       307
   macro avg       0.49      0.57      0.51       307
weighted avg       0.65      0.59      0.59       307



100%|██████████| 16/16 [01:22<00:00,  5.16s/it]



Fold 4 — Accuracy: 0.66, F1 Score: 0.67
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        10
           1       0.00      0.00      0.00         0
           2       0.95      0.55      0.70        69
           3       0.74      0.77      0.76        66
           4       0.64      0.77      0.70        57
           5       0.11      0.50      0.18         2
           6       0.62      0.69      0.65        26
           7       0.75      0.50      0.60        24
           8       0.43      0.67      0.52        27
          10       0.92      1.00      0.96        12
          12       0.17      0.09      0.12        11

    accuracy                           0.66       304
   macro avg       0.54      0.56      0.53       304
weighted avg       0.71      0.66      0.67       304



100%|██████████| 16/16 [01:26<00:00,  5.43s/it]


Fold 5 — Accuracy: 0.54, F1 Score: 0.55
              precision    recall  f1-score   support

           0       0.64      0.56      0.60        16
           1       0.00      0.00      0.00        21
           2       0.50      0.35      0.41        17
           3       0.71      0.27      0.39        44
           4       0.82      0.68      0.74        59
           5       0.03      0.33      0.05         3
           6       0.05      0.20      0.08         5
           7       0.64      0.83      0.72        42
           8       0.53      0.66      0.58        32
          10       0.90      1.00      0.95        18
          12       0.00      0.00      0.00         6

    accuracy                           0.54       263
   macro avg       0.44      0.44      0.41       263
weighted avg       0.60      0.54      0.55       263


Average Accuracy over 5 folds: 0.58 (0.05)
Average F1 Score over 5 folds: 0.58 (0.05)


In [45]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [46]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .690

100%|██████████| 16/16 [00:04<00:00,  3.91it/s]

Accuracy 0.6761565836298933


### Experiment 2: train on 2023 (no change), predict on 2023 (no change)

In [ ]:
df23subset = gdf23[gdf23['change'] == 0].copy()
# class and 2023_class are the same for no change points

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
groups = df23subset['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.56, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.00      0.00      0.00         0
           2       0.10      0.33      0.16        15
           3       0.49      0.56      0.52        64
           4       0.60      0.79      0.68        58
           5       0.83      0.44      0.58        34
           6       0.83      0.39      0.53        51
           7       0.91      0.78      0.84        37
           8       0.66      0.43      0.52        90
          10       0.38      1.00      0.55         8

    accuracy                           0.56       369
   macro avg       0.55      0.54      0.51       369
weighted avg       0.66      0.56      0.58       369


Fold 2 — Accuracy: 0.57, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.76      0.57      0.65        23
           1       0.11      0.05      0.07       

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [32]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .60

Accuracy: 0.63


In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
groups = df23subset['cluster']  # replace with the appropriate grouping column

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.60, F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.70      0.58      0.64        12
           1       0.00      0.00      0.00         0
           2       0.21      0.60      0.32        15
           3       0.54      0.67      0.60        64
           4       0.60      0.69      0.64        58
           5       0.89      0.47      0.62        34
           6       0.89      0.65      0.75        51
           7       0.72      0.78      0.75        37
           8       0.70      0.41      0.52        90
          10       0.42      1.00      0.59         8

    accuracy                           0.60       369
   macro avg       0.57      0.59      0.54       369
weighted avg       0.68      0.60      0.61       369


Fold 2 — Accuracy: 0.68, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.78      0.61      0.68        23
           1       0.00      0.00      0.00       

In [51]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [52]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .760

Accuracy 0.7546628407460545


### Experiment 3: train on 2018 (2018), predict on 2023 (no change)

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        12
           1       0.00      0.00      0.00         0
           2       0.43      0.20      0.27        15
           3       0.46      0.70      0.56        64
           4       0.49      0.72      0.59        58
           5       0.78      0.21      0.33        34
           6       0.63      0.24      0.34        51
           7       1.00      0.41      0.58        37
           8       0.55      0.48      0.51        90
          10       0.67      1.00      0.80         8

    accuracy                           0.50       369
   macro avg       0.57      0.46      0.47       369
weighted avg       0.61      0.50      0.50       369


Fold 2 — Accuracy: 0.45, F1 Score: 0.45
              precision    recall  f1-score   support

           0       0.50      0.20      0.29         5
           1       0.00      0.00      0.00       

In [53]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = gdf18[wavelength23_cols], gdf18['class'].astype(int)

clf.fit(X, y)

RandomForestClassifier()

In [54]:
prediction_probabilities = clf.predict_proba(df23subset[wavelength23_cols])
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(df23subset[wavelength23_cols])
# Calculate accuracy
accuracy = accuracy_score(df23subset['class'], predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .05

Accuracy: 0.54


In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [01:08<00:00,  4.28s/it]



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00        15
           3       0.46      0.41      0.43        64
           4       0.51      0.52      0.51        58
           5       0.50      0.09      0.15        34
           6       0.70      0.61      0.65        51
           7       0.95      0.57      0.71        37
           8       0.46      0.63      0.54        90
           9       0.00      0.00      0.00         0
          10       0.80      1.00      0.89         8

    accuracy                           0.50       369
   macro avg       0.46      0.41      0.41       369
weighted avg       0.55      0.50      0.50       369



100%|██████████| 16/16 [01:10<00:00,  4.44s/it]



Fold 2 — Accuracy: 0.51, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.50      0.60      0.55         5
           1       0.00      0.00      0.00         0
           2       1.00      0.16      0.28        68
           3       0.29      0.65      0.40        31
           4       0.50      0.16      0.24        32
           5       0.51      0.58      0.54        45
           6       0.70      0.64      0.67        22
           7       0.82      0.82      0.82        11
           8       0.57      0.85      0.68        47
          10       1.00      0.79      0.88        19

    accuracy                           0.51       280
   macro avg       0.59      0.52      0.51       280
weighted avg       0.67      0.51      0.50       280



100%|██████████| 16/16 [01:18<00:00,  4.92s/it]



Fold 3 — Accuracy: 0.48, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.27      0.91      0.42        23
           1       0.52      0.79      0.62        19
           2       0.75      0.19      0.30        16
           3       0.82      0.40      0.54        99
           4       0.75      0.25      0.38        60
           5       0.41      0.50      0.45        14
           6       0.20      0.21      0.21        14
           7       0.79      0.62      0.70        24
           8       0.10      0.56      0.18         9
          10       1.00      1.00      1.00        20

    accuracy                           0.48       298
   macro avg       0.56      0.54      0.48       298
weighted avg       0.68      0.48      0.50       298



100%|██████████| 16/16 [01:15<00:00,  4.73s/it]



Fold 4 — Accuracy: 0.54, F1 Score: 0.51
              precision    recall  f1-score   support

           0       0.56      0.71      0.62         7
           1       0.00      0.00      0.00         0
           2       1.00      0.06      0.11        69
           3       0.66      0.85      0.74        61
           4       0.84      0.57      0.68        37
           5       0.00      0.00      0.00         2
           6       0.41      0.46      0.44        26
           7       0.77      0.91      0.83        11
           8       0.32      0.84      0.46        19
          10       1.00      1.00      1.00        12

    accuracy                           0.54       244
   macro avg       0.56      0.54      0.49       244
weighted avg       0.74      0.54      0.51       244



100%|██████████| 16/16 [01:31<00:00,  5.69s/it]


Fold 5 — Accuracy: 0.52, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.67      0.77      0.71        13
           1       0.00      0.00      0.00        21
           2       1.00      0.06      0.11        17
           3       0.53      0.49      0.51        39
           4       0.71      0.32      0.44        31
           5       0.30      1.00      0.46         3
           6       0.11      0.60      0.18         5
           7       0.83      0.83      0.83        30
           8       0.38      0.68      0.49        28
          10       0.89      1.00      0.94        16

    accuracy                           0.52       203
   macro avg       0.54      0.57      0.47       203
weighted avg       0.59      0.52      0.50       203


Average Accuracy (only folds using 2023 test data): 0.51 (0.02)
Average F1 Score: 0.50 (0.00)


In [55]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = gdf18[wavelength23_cols], gdf18['class'].astype(int)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [56]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(df23subset[wavelength23_cols])
print("Accuracy", accuracy_score(df23subset['class'], predictions))

# from 0.08

100%|██████████| 16/16 [00:06<00:00,  2.52it/s]

Accuracy 0.5767575322812052


### Experiment 4: train on 2018 (all) and 2023 (no change), predict on 2018 and 2023 (no change)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        24
           1       0.00      0.00      0.00         0
           2       0.23      0.49      0.31        47
           3       0.42      0.68      0.52       142
           4       0.54      0.71      0.61       123
           5       0.77      0.28      0.41        72
           6       0.80      0.39      0.52       103
           7       0.75      0.54      0.63        74
           8       0.64      0.29      0.40       181
           9       0.58      0.76      0.66        25
          10       0.48      1.00      0.65        16

    accuracy                           0.50       807
   macro avg       0.52      0.51      0.47       807
weighted avg       0.59      0.50      0.50       807


Fold 2 — Accuracy: 0.52, F1 Score: 0.54
              precision    recall  f1-score   support

           0       0.73      0.40      0.52       

In [40]:
df1823 = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols]], ignore_index=True)
df1823

n18 = len(gdf18)
n23 = len(df23subset)


In [61]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df1823.index, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [62]:
train_from_18 = (idx_train < n18).sum()
train_from_23 = (idx_train >= n18).sum()
test_from_18 = (idx_test < n18).sum()
test_from_23 = (idx_test >= n18).sum()

train_total = len(idx_train)
test_total = len(idx_test)

print("Train proportions:")
print(f"df18: {train_from_18 / train_total:.2f}, df23: {train_from_23 / train_total:.2f}")

print("Test proportions:")
print(f"df18: {test_from_18 / test_total:.2f}, df23: {test_from_23 / test_total:.2f}")


Train proportions:
df18: 0.55, df23: 0.45
Test proportions:
df18: 0.55, df23: 0.45


In [63]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .62

Accuracy: 0.65


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:41<00:00, 10.11s/it]



Fold 1 — Accuracy: 0.64, F1 Score: 0.65
              precision    recall  f1-score   support

           0       0.74      0.58      0.65        24
           1       0.00      0.00      0.00         0
           2       0.34      0.66      0.45        47
           3       0.56      0.70      0.62       142
           4       0.66      0.70      0.68       123
           5       0.80      0.44      0.57        72
           6       0.87      0.70      0.77       103
           7       0.86      0.76      0.81        74
           8       0.73      0.50      0.59       181
           9       0.63      0.88      0.73        25
          10       0.59      1.00      0.74        16

    accuracy                           0.64       807
   macro avg       0.61      0.63      0.60       807
weighted avg       0.70      0.64      0.65       807



100%|██████████| 16/16 [02:52<00:00, 10.79s/it]



Fold 2 — Accuracy: 0.66, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.93      0.65      0.76        20
           1       0.00      0.00      0.00         0
           2       0.87      0.64      0.74       192
           3       0.34      0.40      0.37        70
           4       0.69      0.62      0.65        81
           5       0.65      0.69      0.67        91
           6       0.64      0.82      0.72        44
           7       0.81      0.95      0.88        22
           8       0.55      0.65      0.60        97
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.97        39

    accuracy                           0.66       656
   macro avg       0.59      0.58      0.58       656
weighted avg       0.71      0.66      0.68       656



100%|██████████| 16/16 [02:57<00:00, 11.10s/it]



Fold 3 — Accuracy: 0.69, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.75      0.72      0.73        46
           1       0.20      0.05      0.08        38
           2       0.42      0.78      0.55        32
           3       0.88      0.68      0.77       201
           4       0.68      0.80      0.73       125
           5       0.47      0.64      0.55        28
           6       0.43      0.54      0.48        28
           7       0.65      0.77      0.70        48
           8       0.59      0.56      0.57        18
           9       0.00      0.00      0.00         0
          10       1.00      0.98      0.99        41

    accuracy                           0.69       605
   macro avg       0.55      0.59      0.56       605
weighted avg       0.70      0.69      0.68       605



100%|██████████| 16/16 [03:02<00:00, 11.38s/it]



Fold 4 — Accuracy: 0.74, F1 Score: 0.75
              precision    recall  f1-score   support

           0       0.92      0.69      0.79        16
           1       0.00      0.00      0.00         0
           2       0.88      0.64      0.74       148
           3       0.81      0.77      0.79       126
           4       0.75      0.79      0.77        76
           5       0.09      0.25      0.13         4
           6       0.60      0.62      0.61        52
           7       0.64      0.82      0.72        22
           8       0.48      0.82      0.60        38
           9       0.90      0.88      0.89        42
          10       0.96      1.00      0.98        24

    accuracy                           0.74       548
   macro avg       0.64      0.66      0.64       548
weighted avg       0.78      0.74      0.75       548



100%|██████████| 16/16 [03:05<00:00, 11.57s/it]


Fold 5 — Accuracy: 0.64, F1 Score: 0.64
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        28
           1       0.60      0.07      0.12        45
           2       0.68      0.50      0.58        34
           3       0.80      0.72      0.76        82
           4       0.62      0.69      0.66        65
           5       0.10      0.50      0.17         6
           6       0.15      0.50      0.23        10
           7       0.70      0.75      0.73        60
           8       0.55      0.61      0.58        56
           9       0.87      0.69      0.77        48
          10       0.94      1.00      0.97        32

    accuracy                           0.64       466
   macro avg       0.62      0.62      0.58       466
weighted avg       0.70      0.64      0.64       466


Average Accuracy (only folds using 2023 test data): 0.68 (0.04)
Average F1 Score: 0.68 (0.04)


In [64]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [65]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .767

100%|██████████| 16/16 [00:05<00:00,  2.71it/s]

Accuracy 0.7709279688513953


### Experiment 5: train on 2018 (all) and 2023 (no change), predict on 2023 (change). Then train on all 2018 and 2023 and predict on 2023 (all)


In [50]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['change'] == 1) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]

Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        24
           1       0.00      0.00      0.00         0
           2       0.23      0.49      0.31        47
           3       0.42      0.68      0.52       142
           4       0.54      0.71      0.61       123
           5       0.77      0.28      0.41        72
           6       0.80      0.39      0.52       103
           7       0.75      0.54      0.63        74
           8       0.64      0.29      0.40       181
           9       0.58      0.76      0.66        25
          10       0.48      1.00      0.65        16

    accuracy                           0.50       807
   macro avg       0.52      0.51      0.47       807
weighted avg       0.59      0.50      0.50       807

predicted_class
4    37
8     8
3     8
7     6
0     4
1     2
5     1
9     1
Name: count, dtype: int64
1 [0]

Fold 2 — Accuracy: 0.52, F1

In [55]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], df23subset[['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.51, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.48      0.46      0.47        24
           1       0.00      0.00      0.00         0
           2       0.23      0.47      0.31        47
           3       0.42      0.69      0.52       142
           4       0.54      0.72      0.62       123
           5       0.78      0.35      0.48        72
           6       0.81      0.41      0.54       103
           7       0.73      0.50      0.59        74
           8       0.70      0.31      0.44       181
           9       0.59      0.76      0.67        25
          10       0.50      1.00      0.67        16

    accuracy                           0.51       807
   macro avg       0.53      0.51      0.48       807
weighted avg       0.61      0.51      0.52       807


Fold 2 — Accuracy: 0.51, F1 Score: 0.53
              precision    recall  f1-score   support

           0       0.90      0.45      0.60       

In [45]:
# get pseudo-labels for 2023 change class

df23changesubset = gdf23[gdf23['change'] == 1].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

# Ensure the 'class' column is of compatible type (e.g., int)
df23changesubset['class'] = df23changesubset['class'].astype(int)

# Compare predicted and labels before change (lower agreement is better)
matches = df23changesubset['predicted_class'] == df23changesubset['class']

# Compute proportion (i.e., accuracy)
proportion = matches.mean()

print(f"Proportion of matches: {proportion:.2f}")

Proportion of matches: 0.17


In [ ]:
# without spatial CV
df1823 = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols]], ignore_index=True)
df1823

n18 = len(gdf18)
n23 = len(df23subset)


In [67]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df1823.index, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [68]:
train_from_18 = (idx_train < n18).sum()
train_from_23 = (idx_train >= n18).sum()
test_from_18 = (idx_test < n18).sum()
test_from_23 = (idx_test >= n18).sum()

train_total = len(idx_train)
test_total = len(idx_test)

print("Train proportions:")
print(f"df18: {train_from_18 / train_total:.2f}, df23: {train_from_23 / train_total:.2f}")

print("Test proportions:")
print(f"df18: {test_from_18 / test_total:.2f}, df23: {test_from_23 / test_total:.2f}")


Train proportions:
df18: 0.55, df23: 0.45
Test proportions:
df18: 0.55, df23: 0.45


In [69]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .63

Accuracy: 0.65


In [70]:
# get pseudo-labels for 2023 change class

df23changesubset = gdf23[gdf23['change'] == 1].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

# Ensure the 'class' column is of compatible type (e.g., int)
df23changesubset['class'] = df23changesubset['class'].astype(int)

# Compare predicted and true labels
matches = df23changesubset['predicted_class'] == df23changesubset['class']

# Compute proportion (i.e., accuracy)
proportion = matches.mean()

print(f"Proportion of matches: {proportion:.2f}")


Proportion of matches: 0.15


In [71]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(df23changesubset['class'], df23changesubset['predicted_class']))
print(classification_report(df23changesubset['class'], df23changesubset['predicted_class']))


[[ 4  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  3  0  0]
 [ 4  0  8 12 36  8  4  1 10  0  0]
 [ 1  0  3  6 11  1  0  0 11  0  0]
 [ 6  0  1  3 22  0  0  1  1  0  0]
 [ 1  0  0  1  1  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  3  0  0  0  0  0  0]
 [ 0  0  0  7 80  0  0 18  9  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  2]]
              precision    recall  f1-score   support

           0       0.25      0.29      0.27        14
           1       0.00      0.00      0.00         3
           2       0.67      0.10      0.17        83
           3       0.20      0.18      0.19        33
           4       0.14      0.65      0.24        34
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         4
           9       1.00      0.01      0.02       115
        

In [72]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols], df23changesubset[['class']+wavelength23_cols]], ignore_index=True)

In [73]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = dfallsemisuperivised[wavelength23_cols], dfallsemisuperivised['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [74]:
# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .65

Accuracy: 0.67


TABPFN

In [57]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['change'] == 1) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]


100%|██████████| 16/16 [02:41<00:00, 10.11s/it]



Fold 1 — Accuracy: 0.65, F1 Score: 0.66
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        24
           1       0.00      0.00      0.00         0
           2       0.32      0.64      0.43        47
           3       0.57      0.73      0.64       142
           4       0.67      0.71      0.69       123
           5       0.82      0.46      0.59        72
           6       0.87      0.71      0.78       103
           7       0.85      0.77      0.81        74
           8       0.74      0.49      0.59       181
           9       0.64      0.92      0.75        25
          10       0.62      1.00      0.76        16

    accuracy                           0.65       807
   macro avg       0.62      0.64      0.61       807
weighted avg       0.70      0.65      0.66       807



100%|██████████| 16/16 [02:11<00:00,  8.19s/it]


predicted_class
4    36
7    10
3     7
8     7
2     4
1     2
5     1
Name: count, dtype: int64
1 [0]


...

predicted_class
4    37
8    10
3     7
7     3
0     2
5     1
Name: count, dtype: int64
4 [1]


100%|██████████| 16/16 [03:02<00:00, 11.42s/it]



Fold 5 — Accuracy: 0.63, F1 Score: 0.63
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        28
           1       0.50      0.04      0.08        45
           2       0.69      0.53      0.60        34
           3       0.81      0.71      0.75        82
           4       0.61      0.66      0.63        65
           5       0.07      0.33      0.11         6
           6       0.14      0.50      0.22        10
           7       0.66      0.75      0.70        60
           8       0.56      0.61      0.58        56
           9       0.86      0.67      0.75        48
          10       0.94      1.00      0.97        32

    accuracy                           0.63       466
   macro avg       0.61      0.60      0.57       466
weighted avg       0.68      0.63      0.63       466



100%|██████████| 16/16 [02:39<00:00,  9.94s/it]

predicted_class
4     39
8      7
7      7
3      3
10     2
5      2
Name: count, dtype: int64

Average Accuracy (only folds using 2023 test data): 0.68 (0.04)
Average F1 Score: 0.68 (0.04)


In [58]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], df23subset[['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:56<00:00, 11.00s/it]



Fold 1 — Accuracy: 0.65, F1 Score: 0.66
              precision    recall  f1-score   support

           0       0.82      0.58      0.68        24
           1       0.00      0.00      0.00         0
           2       0.35      0.64      0.45        47
           3       0.57      0.71      0.63       142
           4       0.65      0.70      0.67       123
           5       0.78      0.44      0.57        72
           6       0.89      0.75      0.81       103
           7       0.82      0.72      0.76        74
           8       0.75      0.52      0.62       181
           9       0.61      0.88      0.72        25
          10       0.59      1.00      0.74        16

    accuracy                           0.65       807
   macro avg       0.62      0.63      0.61       807
weighted avg       0.70      0.65      0.66       807



100%|██████████| 16/16 [03:06<00:00, 11.64s/it]



Fold 2 — Accuracy: 0.66, F1 Score: 0.67
              precision    recall  f1-score   support

           0       0.93      0.65      0.76        20
           1       0.00      0.00      0.00         0
           2       0.86      0.65      0.74       192
           3       0.34      0.43      0.38        70
           4       0.72      0.64      0.68        81
           5       0.62      0.66      0.64        91
           6       0.62      0.80      0.70        44
           7       0.80      0.91      0.85        22
           8       0.55      0.61      0.58        97
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.97        39

    accuracy                           0.66       656
   macro avg       0.59      0.57      0.57       656
weighted avg       0.70      0.66      0.67       656



100%|██████████| 16/16 [03:09<00:00, 11.81s/it]



Fold 3 — Accuracy: 0.70, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.74      0.70      0.72        46
           1       0.00      0.00      0.00        38
           2       0.40      0.78      0.53        32
           3       0.88      0.71      0.79       201
           4       0.69      0.82      0.75       125
           5       0.49      0.61      0.54        28
           6       0.44      0.57      0.50        28
           7       0.71      0.75      0.73        48
           8       0.60      0.50      0.55        18
           9       0.00      0.00      0.00         0
          10       1.00      0.98      0.99        41

    accuracy                           0.70       605
   macro avg       0.54      0.58      0.55       605
weighted avg       0.70      0.70      0.69       605



100%|██████████| 16/16 [03:13<00:00, 12.07s/it]



Fold 4 — Accuracy: 0.74, F1 Score: 0.75
              precision    recall  f1-score   support

           0       0.91      0.62      0.74        16
           1       0.00      0.00      0.00         0
           2       0.87      0.66      0.75       148
           3       0.83      0.76      0.79       126
           4       0.72      0.78      0.75        76
           5       0.09      0.25      0.13         4
           6       0.63      0.65      0.64        52
           7       0.62      0.73      0.67        22
           8       0.48      0.82      0.60        38
           9       0.88      0.88      0.88        42
          10       0.96      1.00      0.98        24

    accuracy                           0.74       548
   macro avg       0.63      0.65      0.63       548
weighted avg       0.78      0.74      0.75       548



100%|██████████| 16/16 [03:11<00:00, 11.98s/it]


Fold 5 — Accuracy: 0.69, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.79      0.82      0.81        28
           1       0.67      0.09      0.16        45
           2       0.67      0.53      0.59        34
           3       0.81      0.72      0.76        85
           4       0.76      0.81      0.79       104
           5       0.12      0.50      0.20         8
           6       0.15      0.50      0.23        10
           7       0.75      0.78      0.76        67
           8       0.62      0.70      0.66        63
           9       0.84      0.67      0.74        48
          10       0.94      1.00      0.97        34

    accuracy                           0.69       526
   macro avg       0.65      0.65      0.61       526
weighted avg       0.74      0.69      0.69       526


Average Accuracy: 0.69 (0.03)
Average F1 Score: 0.69 (0.03)


In [75]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [76]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

100%|██████████| 16/16 [00:05<00:00,  2.75it/s]

Accuracy 0.7696301103179753


In [77]:
# get pseudo-labels for 2023 change class

df23changesubset = gdf23[gdf23['change'] == 1].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

# Ensure the 'class' column is of compatible type (e.g., int)
df23changesubset['class'] = df23changesubset['class'].astype(int)

# Compare predicted and true labels
matches = df23changesubset['predicted_class'] == df23changesubset['class']

# Compute proportion (i.e., accuracy)
proportion = matches.mean()

print(f"Proportion of matches: {proportion:.2f}")


100%|██████████| 16/16 [00:04<00:00,  3.57it/s]

Proportion of matches: 0.22


In [78]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(df23changesubset['class'], df23changesubset['predicted_class']))
print(classification_report(df23changesubset['class'], df23changesubset['predicted_class']))


[[ 4  0  0  0  0  0  0  0  0  0 10]
 [ 0  2  0  0  0  0  0  0  1  0  0]
 [ 2  1 19 16 33  6  1  2  3  0  0]
 [ 0  0  5 13 11  1  0  0  3  0  0]
 [ 2  0  0  4 22  0  0  5  1  0  0]
 [ 0  1  1  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  3  0  0  0  0  0  0]
 [ 0  0  0  2 87  0  0 16 10  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2]]
              precision    recall  f1-score   support

           0       0.50      0.29      0.36        14
           1       0.50      0.67      0.57         3
           2       0.76      0.23      0.35        83
           3       0.36      0.39      0.38        33
           4       0.14      0.65      0.23        34
           5       0.00      0.00      0.00         3
           6       0.50      1.00      0.67         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00       115
        

In [79]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols], df23changesubset[['class']+wavelength23_cols]], ignore_index=True)

In [80]:
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = dfallsemisuperivised[wavelength23_cols], dfallsemisuperivised['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)


ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [81]:
# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .786

100%|██████████| 16/16 [00:06<00:00,  2.35it/s]

Accuracy 0.7806757557794902


### Experiment 6: train on 2023 (stable), predict on 2018 (all labelled)

In [82]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)

clf.fit(X, y)

RandomForestClassifier()

In [85]:
prediction_probabilities = clf.predict_proba(gdf18[wavelength23_cols])
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(gdf18[wavelength23_cols])
# Calculate accuracy
accuracy = accuracy_score(gdf18['class'], predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .04

Accuracy: 0.44


In [86]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [89]:
# Get the predicted class labels
predicted_labels = clf.predict(gdf18[wavelength23_cols])
# Calculate accuracy
accuracy = accuracy_score(gdf18['class'], predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .05

Accuracy: 0.50


### Experiment 7: train on 2018 (all) and 2023 (all), predict on 2023 (no change)

In [90]:
# use 2018 class for 2023 whereever there is no change
gdf23.loc[gdf23['change'] == 0, '2023_class'] = gdf23['class']
gdf23.dropna(subset=['2023_class'], inplace=True)

In [91]:
df1823 = pd.concat([gdf18[['class']+wavelength23_cols],
                    gdf23[['2023_class']+wavelength23_cols].rename(columns={'2023_class': 'class'})]
, ignore_index=True)
df1823

n18 = len(gdf18)
n23 = len(gdf23)


In [92]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df1823.index, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [93]:
train_from_18 = (idx_train < n18).sum()
train_from_23 = (idx_train >= n18).sum()
test_from_18 = (idx_test < n18).sum()
test_from_23 = (idx_test >= n18).sum()

train_total = len(idx_train)
test_total = len(idx_test)

print("Train proportions:")
print(f"df18: {train_from_18 / train_total:.2f}, df23: {train_from_23 / train_total:.2f}")

print("Test proportions:")
print(f"df18: {test_from_18 / test_total:.2f}, df23: {test_from_23 / test_total:.2f}")


Train proportions:
df18: 0.49, df23: 0.51
Test proportions:
df18: 0.51, df23: 0.49


In [94]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .59

Accuracy: 0.62


In [95]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [96]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .748

100%|██████████| 16/16 [00:06<00:00,  2.43it/s]

Accuracy 0.7291049199762892
